# Experimental data (15%Tm,  85%Yb) 

In [1]:
C2 = 15

blue15 = [3, 7.5, 2.5*10, 9*10, 2.3*10**2, 2.5*10**2, 5.4*10**2, 7.5*10**2, 9*10**2, 1*10**3, 1.2*10**3, 1.4*10**3]
P980_blue15 = [4.4*10**3, 5*10**3, 6*10**3, 7.5*10**3, 9.1*10**3, 1.1*10**4, 1.4*10**4, 1.7*10**4, 2*10**4, 2.3*10**4, 2.7*10**4, 3*10**4]

NIR15 = [3*10, 6*10, 1.4*10**2, 6*10**2, 8*10**2, 1.4*10**3, 2*10**3, 2.6*10**3, 3.5*10**3, 4*10**3, 4.5*10**3, 4.9*10**3, 5*10**3, 5*10**3, 5*10**3]
P980_NIR15 = [1.2*10**3, 1.8*10**3, 2.8*10**3, 4.4*10**3, 5*10**3, 6*10**3, 7.5*10**3, 9.1*10**3, 1.1*10**4, 1.4*10**4, 1.7*10**4, 2*10**4, 2.3*10**4, 2.7*10**4, 3*10**4]


P980_NIR = P980_NIR15

P980_blue = P980_blue15

NIR_exp = NIR15

blue_exp = blue15

# define global variables

In [2]:


epsilon = 1e-1 # something to change

learning_rate1 = 5 # something to change
    
learning_rate2 = 5 # something to change

small_threshold = 1e-2 # something to change

c1_range = [(50, 300)] 

c2_range = [(1, 3)]

c3_range = [(1, 4)]

c4_range = [(1, 5)]

k31_range = [(1, 6)]

k41_range = [(1, 7)]

k51_range = [(1, 8)]

parameter_ranges = c1_range + c2_range + c3_range + c4_range + k31_range + k41_range + k51_range


dic = {'c1': 0, 'c2': 1, 'c3': 2, 'c4': 3, 'k31': 4, 'k41': 5, 'k51': 6}


x = C2

In [3]:
import numpy as np
import json
from tqdm import tqdm
from scipy.optimize import minimize
from scipy.integrate import odeint


a21 = 1
a31 = 0.27
a32 = 0.73

a41 = 0.18
a42 = 0.24
a43 = 0.58

a51 = 0.24
a52 = 0.23
a53 = 0.2
a54 = 0.33

W2 = 63000
W3 = 20000
W4 = 15000
W5 = 33000
Ws2 = 8000



def system(state, t, x, P980, c1, c2, c3, c4, k31, k41, k51):

    ns2, n1, n2, n3, n4, n5 = state

    #ns2
    ms2 = 1.23*P980*(37*(100-x)-ns2) - Ws2*ns2 - (c1*n1+c2*n2+c3*n3+c4*n4)*ns2  # ns1 = total_Yb - ns2

    # n1
    m1 = -c1*n1*ns2 + a21*W2*n2 + a31*W3*n3 + a41*W4*n4 + a51*W5*n5 - k41*n1*n4 - k31*n1*n3 - k51*n5*n1

    # n2
    m2 = c1*n1*ns2 - c2*n2*ns2 - a21*W2*n2 + a32*W3*n3 + a42*W4*n4 + a52*W5*n5 + k41*n1*n4 + 2*k31*n1*n3

    # n3
    m3 = c2*n2*ns2 - c3*n3*ns2 - (a31+a32)*W3*n3 + a43*W4*n4 + a53*W5*n5 + 2*k51*n1*n5 + k41*n1*n4 - k31*n1*n3

    # n4
    m4 = c3*n3*ns2 - c4*n4*ns2 - (a43+a42+a41)*W4*n4 + a54*W5*n5 - k41*n1*n4

    # n5
    m5 = c4*n4*ns2 - (a54+a53+a52+a51)*W5*n5 - k51*n1*n5


    return [ms2, m1, m2, m3, m4, m5]


def compute_blue_for_x(x, P980, c1, c2, c3, c4, k31, k41, k51):


    y = 37 * x
    state0 = [0, y, 0, 0, 0, 0]
    t = np.arange(0.0, 0.001, 0.000001)
    state = odeint(system, state0, t, args=(x, P980, c1, c2, c3, c4, k31, k41, k51))
    blue_total = a41 * W4 * state[:, 4][-1] + a52 * W5 * state[:, 5][-1]

    return blue_total

def compute_NIR_for_x(x, P980, c1, c2, c3, c4, k31, k41, k51):

    y = 37 * x
    state0 = [0, y, 0, 0, 0, 0]
    t = np.arange(0.0, 0.001, 0.000001)
    state = odeint(system, state0, t, args=(x, P980, c1, c2, c3, c4, k31, k41, k51))
    NIR = a31 * W3 * state[:, 3][-1]

    return NIR

# cost function(SSE)

In [4]:
def cost_function(params):
    
    c1, c2, c3, c4, k31, k41, k51 = params
    
    cost = 0
    
    for P980, exp_nir in zip(P980_NIR, NIR_exp):
        nir = compute_NIR_for_x(x, P980, c1, c2, c3, c4, k31, k41, k51)
        cost_NIR = ((nir - exp_nir)/exp_nir)**2
        cost += cost_NIR
        
        
    for P980, exp_blue in zip(P980_blue, blue_exp):
        blue = compute_blue_for_x(x, P980, c1, c2, c3, c4, k31, k41, k51)
        cost_blue = ((blue - exp_blue)/exp_blue)**2
        cost += cost_blue    

    return cost / 10000

# Numerical difference: central difference

In [5]:
def gradient(current_params, index):
    
    
    epsilon = 1e-3
    
    params_plus = current_params.copy()
    params_minus = current_params.copy()
    params_plus[index] += epsilon
    params_minus[index] -= epsilon
    grad = (cost_function(params_plus) - cost_function(params_minus)) / (2 * epsilon)
        
    return grad

# There are four functions defined below:

`move_through1`

`move_through2`

`move_through3`

`move_through4`

They can make the `grad` pass through the peak from two directions, and pass through the valley from two directions, by moving a tiny proper step each time. 

In [6]:
def move_through1(grad, current_params, index):
    
    while grad<small_threshold:
        
        current_params[index] +=  1e-1    
        grad = gradient(current_params, index)
        
    return current_params

In [7]:
def move_through2(grad, current_params, index):
    

    while grad > small_threshold*-1:
        
        current_params[index] -=  1e-1  
        grad = gradient(current_params, index)
        
    return current_params 

In [8]:
def move_through3(grad, current_params, index):
    
    while grad<small_threshold:
        
        current_params[index] -=  1e-1    
        grad = gradient(current_params, index)
        
    return current_params

In [9]:
def move_through4(grad, current_params, index):
    

    while grad > small_threshold*-1:
        
        current_params[index] +=  1e-1  
        grad = gradient(current_params, index)
        
    return current_params 


# `gradient_descent_boosting1`

Firstly descent, after reaches the valley, then boosting.

In [10]:
# input parameters: [c1, c2, c3, c4, k31, k41, k51], and p: c1...

def gradient_descent_boosting1(params, p):
    

   
    
    cost1 = []
    
    All_min1 = []  # To store the minimum cost and associated parameters
    
    index = dic[p]
    
    # Initialize the parameter values
    current_params = params.copy()

    while True:
        
          
        boosting = False  # Flag to switch between gradient descent and boosting

        # Check if current_params are within the specified parameter ranges
        
        descent=0
        while not boosting:
            
                descent+=1
            
                grad = gradient(current_params, index)
                print('\n')
                print('Now descent')
                
                print(f'gradient with respect to {p} = {grad}')
                
                
                if grad < 0:
                    flag = -1
                    print(f'grad={grad} <0, move forward')
                    
                else:
                    flag = 1
                    print(f'grad={grad} >0, move backward')

                # Update parameters using gradient descent
                m = current_params[index]
                print(f'before update, this parameter={current_params[index]}')
                current_params[index] -= learning_rate1 * grad
                print(f'after update, this parameter={current_params[index]}')
                print(f'difference = {abs(current_params[index]-m)}')
                
                

                # Check if a valley (gradient close to zero) is reached
                if abs(grad) < small_threshold:
                    # Calculate the cost with the newest parameters
                    print(f'abs_grad={abs(grad)}<{small_threshold}')
                    valley = cost_function(current_params)
                    cost1.append(valley)
                    All_min1.append({'min values': valley, 'parameters': current_params.copy()})
                    boosting = True  # Switch to gradient boosting phase
                    
                    break

     
        (min_val, max_val) = parameter_ranges[index]
        current_val = current_params[index]
        if not (min_val <= current_val <= max_val):
            print(f'This parameter {p} now is {current_val}, which reaches its bound')
            break  


        # Perform gradient boosting until a peak is reached (gradient close to zero)
        
        print('\n')
        print(f'after {descent} descent times, switch from descent to boosting')
        
        boost_times=0
        
        while boosting:
            
            boost_times+=1
            
            if flag == -1:
                current_params=move_through1(grad, current_params, index)
                
            elif flag == 1:
                current_params=move_through2(grad, current_params, index)
                
                
            # Update parameters using boosting (e.g., gradient boosting or another boosting algorithm)
            
            print('pass throuth the valley, now boosting')
    
            grad = gradient(current_params, index)
            
            current_params[index] += learning_rate2 * grad

            # Check if a peak (gradient close to zero) is reached
            if abs(grad) < small_threshold:
                print(f'after {boost_times} boost times, now reaches the peak, switch to descent')
                break
                
                

        (min_val, max_val) = parameter_ranges[index]
        current_val = current_params[index]
        if not (min_val <= current_val <= max_val):
            print(f'This parameter {p} now is {current_val}, which reaches its bound')
            break  

    return All_min1


# `gradient_descent_boosting2`

Firstly boosting, after reaches the peak, then descent.

In [11]:
def gradient_descent_boosting2(params, p):
    
    All_min2 = []  # To store the minimum cost and associated parameters
    
    index = dic[p]
    
    cost2 = []
        

    # Initialize the parameter values
    current_params = params.copy()

    while True:
        
          
        boosting = True  # Start with gradient boosting (climbing)
        peak = None  # Placeholder for the peak

        # Check if current_params are within the specified parameter ranges
      
        boost_times=0
        
        while boosting:
            
            boost_times+=1
            
            
            print('Now boosting')
            grad = gradient(current_params, index)
            
            if grad > 0:
                flag = 1  # Moving in the positive direction (climbing)
                print(f'grad={grad} >0, move forward')
            else:
                flag = -1  # Moving in the negative direction (descending)
                print(f'grad={grad} <0, move backward')
                
                # Update parameters using gradient descent
            m = current_params[index]
            print(f'before update, this parameter={current_params[index]}')
            current_params[index] += learning_rate2 * grad
            print(f'after update, this parameter={current_params[index]}')
            print(f'difference = {abs(current_params[index]-m)}')


            # Check if a peak (gradient close to zero or negative) is reached
            if abs(grad) < small_threshold:
                
                print(f'after {boost_times} boost times, now reaches the peak, switch to descent')
                # Record the cost with the current parameters as a potential peak
                boosting = False  # Switch to gradient descent phase
                
                break
                
                
        (min_val, max_val) = parameter_ranges[index]
        current_val = current_params[index]
        if not (min_val <= current_val <= max_val):
            print(f'This parameter {p} now is {current_val}, which reaches its bound')
            break  

        # Perform gradient descent until a valley is reached (gradient close to zero)
        
        print('\n')
        print('Switch from boosting to descent')
        
        descent = 0
        while not boosting:
            
            descent+=1
            
            if flag == 1:
                current_params=move_through4(grad, current_params, index)
                
            elif flag == -1:
                current_params=move_through3(grad, current_params, index)
                
            print('pass throuth the peak, now descent')

            
            grad = gradient(current_params, index)
            print('\n')
            print(f'gradient with respect to {p} = {grad}')
    

            # Update parameters using gradient descent
            current_params[index] -= learning_rate1 * grad
            

            # Check if a valley (gradient close to zero) is reached
            if abs(grad) < small_threshold:
               
                print(f'abs_grad={abs(grad)}<{small_threshold}')
                valley = cost_function(current_params)
                cost2.append(valley)
                All_min2.append({'min values': valley, 'parameters': current_params.copy()})
                boosting = True  # Switch to gradient boosting phase
                
                break

                
        (min_val, max_val) = parameter_ranges[index]
        current_val = current_params[index]
        if not (min_val <= current_val <= max_val):
            print(f'This parameter {p} now is {current_val}, which reaches its bound')
            break  

    return All_min2


# Give the initial values of the seven parameters, then take one as an example. (eg. `c1`)

In [12]:
initial_params = [2, -1, 1, 1.5, -3, 1.5, -2] 

params = initial_params

p= 'c1'

gradient_descent_boosting1(params, p)



Now descent
gradient with respect to c1 = 1.1116386829934655
grad=1.1116386829934655 >0, move backward
before update, this parameter=2
after update, this parameter=-3.5581934149673273
difference = 5.558193414967327


Now descent
gradient with respect to c1 = -21.843846540757994
grad=-21.843846540757994 <0, move forward
before update, this parameter=-3.5581934149673273
after update, this parameter=105.66103928882265
difference = 109.21923270378997


Now descent
gradient with respect to c1 = -0.02391819949942864
grad=-0.02391819949942864 <0, move forward
before update, this parameter=105.66103928882265
after update, this parameter=105.78063028631979
difference = 0.11959099749714142


Now descent
gradient with respect to c1 = -0.023859650611024463
grad=-0.023859650611024463 <0, move forward
before update, this parameter=105.78063028631979
after update, this parameter=105.89992853937491
difference = 0.11929825305512054


Now descent
gradient with respect to c1 = -0.023804930990634077
gra

KeyboardInterrupt: 